In [1]:
import os
from dotenv import load_dotenv
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper


In [2]:
from discussion_agents.cog.prompts.critic import (
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    CRITIC_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
)
from discussion_agents.cog.agent.critic import CriticAgent

In [3]:
from discussion_agents.cog.prompts.critic import (
    CRITIC_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_DIRECT,
    CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC,
)

In [4]:
load_dotenv()
GOOGLE_API =  os.environ.get('google_api')
GPT_API =  os.environ.get('gpt_api')

In [5]:
question = 'Janet hires six employees. Four of them are warehouse workers who make $15/hour, and the other two are managers who make $20/hour. Janet has to pay 10% of her workers\' salaries in FICA taxes. If everyone works 25 days a month and 8 hours a day, how much does Janet owe total for their wages and taxes for one month?'
llm = ChatOpenAI(openai_api_key=GPT_API)
search = GoogleSearchAPIWrapper(google_api_key = GOOGLE_API, google_cse_id = '7222942e200c847ac')
agent = CriticAgent(llm=llm, search = search)
answer,log = agent.generate(question=question, examples = GSM8K_FEWSHOT_EXAMPLES_DIRECT, prompt=CRITIC_INSTRUCTION_GSM8K, critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC, critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K)

In [6]:
answer

"total_owed\n```\n\nThis code accurately calculates the total amount Janet owes for her employees' wages and taxes for one month. The total amount comes out to $22000, which is the correct answer."

In [7]:
log

"The initial code provided seems to have some issues in the calculations. Let's break down the corrections with a better solution:\n\n```python\n# Define the wages for the workers\nwarehouse_wage = 15\nmanager_wage = 20\n\n# Define the number of employees in each category\nnum_warehouse_workers = 4\nnum_managers = 2\n\n# Calculate the total number of hours worked in a month\nnum_days = 25\nhours_per_day = 8\ntotal_hours = num_days * hours_per_day\n\n# Calculate the total wages for the workers\nwarehouse_wages = num_warehouse_workers * warehouse_wage * total_hours\nmanager_wages = num_managers * manager_wage * total_hours\ntotal_wages = warehouse_wages + manager_wages\n\n# Calculate the total FICA taxes owed\nfica_rate = 0.10\ntotal_fica_tax = total_wages * fica_rate\n\n# Calculate the total amount owed\ntotal_owed = total_wages + total_fica_tax\nanswer = total_owed\n```\n\nThis code accurately calculates the total amount Janet owes for her employees' wages and taxes for one month. The 